In [1]:
!module load CUDA/11.8.0
# sentence_transformers==2.2.2
# chromadb==0.4.5  the newest version conflits with the version of numpy
# langchain==0.0.266


The following have been reloaded with a version change:
  1) CUDA/12.3.0 => CUDA/11.8.0



LLamaparse is not recommended for the Llama2, because this package will overwrite the environment supporting the Llama2.

In [2]:
import torch
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

from auto_gptq import AutoGPTQForCausalLM
from transformers import LlamaForCausalLM, LlamaTokenizer,AutoTokenizer, TextStreamer, pipeline 

import PyPDF2
import requests
import io
import fitz

In [3]:
import re
from fuzzywuzzy import fuzz
import pandas as pd
import csv

In [4]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

# Llama2

In [5]:
model_name_or_path = "TheBloke/Llama-2-70B-chat-GPTQ"
model_basename = "model"

tokenizer_70B = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model_70B = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    quantize_config=None,
)

INFO - The layer lm_head is not quantized.


  0%|          | 0/3043 [00:00<?, ?w/s]

# presetting prompt

In [6]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant in a Wealth Management Company. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

streamer_70B = TextStreamer(tokenizer_70B, skip_prompt=True, skip_special_tokens=True)
text_pipeline = pipeline(
    "text-generation",
    model=model_70B,
    tokenizer=tokenizer_70B,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer_70B,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."
template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCausalLM', 'MvpForCausalLM', 'OpenLlam

load INSTRUCTOR_Transformer
max_seq_length  512


# file accessing

In [7]:

def fileaccessing_builddb(url_path):
    pdf_texts = []
    response = requests.get(url_path)
    pdf_data = response.content
    pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_data))
    for i in range(len(pdf_reader.pages)):
        text = pdf_reader.pages[i].extract_text()
        
        text = text.replace('\xa0', ' ')
        docu = Document(
                page_content=text,
                metadata={},
            )
        pdf_texts.append(docu)
    return pdf_texts


def fileaccessing_builddb_pageselection(url_path, search_text_thresholds):
    pdf_texts = []
    try:
        response = requests.get(url_path)
        pdf_data = response.content
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_data))
    except PyPDF2._reader.PdfReadError:
        HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \ (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'}
        response = requests.get(url_path, headers=HEADERS)
        pdf_data = response.content
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_data))

    for i in range(len(pdf_reader.pages)):
        text = pdf_reader.pages[i].extract_text()
        text = text.replace('\xa0', ' ')

        highest_threshold = 0
        s = 0
        for search_text, threshold in search_text_thresholds:
            s += threshold
            if search_text.lower() in text:
                highest_threshold += 100
            else:
                similarity = fuzz.partial_ratio(search_text.lower(), text)
                if similarity >= threshold:
                    highest_threshold += max(threshold, similarity)

        # If the highest_threshold exceeds the sum of search_text_thresholds,
        # add the document to pdf_texts
        if highest_threshold > s:
            docu = Document(
                page_content=text,
                metadata={},
            )
            pdf_texts.append(docu)

    return pdf_texts


In [8]:
def Quarters_cal(currentQ = "2Q23"):
    q = int(currentQ[0])
    year = int(currentQ[2:4])
    quarters = [currentQ]
    # the previous one quarter
    if q == 1:
        q = 4
        year -= 1
    else:
        q -= 1
    time = f"{q}Q{year}"
    quarters.append(time)
    # the quarter in the past year
    q = int(currentQ[0])
    year = int(currentQ[2:4])
    quarters.append(f"{q}Q{year-1}")
    return quarters

In [9]:
def past_6Q(currentQ = "2Q23"):
    quarters = [currentQ]
    q = int(currentQ[0])
    year = int(currentQ[2:4])
    for i in range(6):
        if q == 1:
            q = 4
            year -= 1
        else:
            q -= 1
        if year < 0:
            year = 100 + year
        time = f"{q}Q{year}"
        quarters.append(time)
    return quarters

## Morgan Stanley

In [10]:
MSlinks = {"4Q23": "https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup4q2023/finsup4q2023.pdf",\
        "2Q23": "https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup2q2023/finsup2q2023.pdf", \
        "3Q22": "https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup3q2022/finsup3q2022.pdf"}

In [11]:
# guessing metrics names
texts = fileaccessing_builddb(MSlinks["2Q23"])
ids = [str(i) for i in range(1, len(texts) + 1)]
MSDB = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    ids=ids
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=MSDB.as_retriever(search_kwargs={"k": 2}), # change database
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)
query  = "I am interested into the values in Wealth management field about asset under management, the change of asset \
and related subsets. please give me the metrics names in the file which will interest me"
result = qa_chain(query)
MSDB._collection.delete(ids=ids)

/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Sure! Based on your interest in values in the wealth management field, specifically asset under management, changes in assets, and related subsets, here are some metrics you might find useful from the provided text:

* Total client assets
* Net new assets
* Advisor-led client assets
* Fee-based client assets
* Self-directed assets
* Stock plan unvested assets

These metrics can be found in the "Wealth Management Metrics" section of the text.


In [12]:
# data extraction 
MS_search_words_with_threshold =[('Wealth Management Metrics',50),('Advisor‐led channel',50),('Financial Information',50),('Asset Management',50)]
source_docu = []
results = []
for index, link in MSlinks.items():
    print(link)
    texts = fileaccessing_builddb_pageselection(link, search_text_thresholds = MS_search_words_with_threshold)
    quarters = Quarters_cal(currentQ = index)
    print(len(texts))
    ids = [str(i) for i in range(1, len(texts) + 1)]
    MSDB = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        ids=ids
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=MSDB.as_retriever(search_kwargs={"k": 2}), 
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )
    query  = "Create a table to export values about Total client assets, Net new assets, \
    Advisor‐led client assets, Fee‐based client assets and flows , Self‐directed assets about Wealth Management of "
    for quarter in quarters:
        query += f"{quarter}, "
    query += " In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE! \
    The Row names are QUARTERS and the Column names are METRICS!"
    result = qa_chain(query)
    if "I don't know" in result['result']:
        print(texts)
    results.append(result['result'])
    source_docu.append(result['source_documents'])
    print("count before", MSDB._collection.count())
    MSDB._collection.delete(ids=ids)
    print("count after", MSDB._collection.count())
    # check whether the database is cleaned or not.

https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup4q2023/finsup4q2023.pdf
2
 Sure! Here's your table:

| QUARTERS | Total Client Assets B ($) | Net New Assets B ($) | Advisor-Led Client Assets B ($) | Fee-Based Client Assets B ($) | Fee-Based Flows B ($) | Self-Directed Assets B ($) |
| --- | --- | --- | --- | --- | --- | --- |
| 4Q23 | 5,129 | 47.5 | 3,979 | 1,983 | 41.6 | 1,150 |
| 3Q23 | 4,798 | 35.7 | 3,755 | 1,857 | 23.1 | 1,043 |
| 4Q22 | 4,187 | 51.6 | 3,392 | 1,678 | 22.0 | 795 |

Note that I used the information from the "Wealth Management Metrics" section of the provided text to fill out the table. Let me know if there's anything else I can help with!
count before 2
count after 0
https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup2q2023/finsup2q2023.pdf
3
 Sure, here's your table:

| QUARTERS | Total Client Assets B | Net New Assets B | Advisor-Led Client Assets B | Fee-Based Client Assets B | Fee-Based Flows B | Self-Directed Assets

In [13]:
for i in results:
    print(i)

  Sure! Here's your table:

| QUARTERS | Total Client Assets B ($) | Net New Assets B ($) | Advisor-Led Client Assets B ($) | Fee-Based Client Assets B ($) | Fee-Based Flows B ($) | Self-Directed Assets B ($) |
| --- | --- | --- | --- | --- | --- | --- |
| 4Q23 | 5,129 | 47.5 | 3,979 | 1,983 | 41.6 | 1,150 |
| 3Q23 | 4,798 | 35.7 | 3,755 | 1,857 | 23.1 | 1,043 |
| 4Q22 | 4,187 | 51.6 | 3,392 | 1,678 | 22.0 | 795 |

Note that I used the information from the "Wealth Management Metrics" section of the provided text to fill out the table. Let me know if there's anything else I can help with!
  Sure, here's your table:

| QUARTERS | Total Client Assets B | Net New Assets B | Advisor-Led Client Assets B | Fee-Based Client Assets B | Fee-Based Flows B | Self-Directed Assets B |
| --- | --- | --- | --- | --- | --- | --- |
| 2Q23 | 4,885 | 89.5 | 3,784 | 1,856 | 22.7 | 1,101 |
| 1Q23 | 4,558 | 109.6 | 3,582 | 1,769 | 21.1 | 976 |
| 2Q22 | 4,246 | 52.9 | 3,427 | 1,717 | 24.8 | 819 |

Note: The v

In [46]:
# Save the raw tables
pattern = r'\| QUARTERS \|.*?\|\n(.*?)\n\n'
MSdataframes = []
for i in results:
    match = re.search(pattern, i, re.DOTALL)
    if match:
        table_str = match.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        
        headers = [header.strip() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
    
        df = pd.DataFrame(data, columns=headers)
        MSdataframes.append(df)
        

In [47]:
# Merge tables
pattern = r'\| QUARTERS \|.*?\|\n(.*?)\n\n'
tables = []

for i in results:
    match = re.search(pattern, i, re.DOTALL)
    if match:
        table_str = match.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        # Extract headers (first row) and data (remaining rows)
        headers = [header.strip().replace('B ($)', '(B)').replace(' B', ' (B)').lower() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
combined_df = pd.concat(tables, ignore_index=True)
combined_df.drop(combined_df.tail(2).index, inplace=True)
subset_df = combined_df[combined_df['quarters'].isin(past_6Q(currentQ = "4Q23"))]
subset_df = subset_df.set_index('quarters')
subset_df = subset_df.reindex(index=past_6Q(currentQ = "4Q23")) 
MSdataframes.append(subset_df)
print(subset_df)
        

         total client assets (b) net new assets (b)  \
quarters                                              
4Q23                       5,129               47.5   
3Q23                       4,798               35.7   
2Q23                       4,885               89.5   
1Q23                       4,558              109.6   
4Q22                       4,187               51.6   
3Q22                       4,134               64.8   
2Q22                       4,246               52.9   

         advisor-led client assets (b) fee-based client assets (b)  \
quarters                                                             
4Q23                             3,979                       1,983   
3Q23                             3,755                       1,857   
2Q23                             3,784                       1,856   
1Q23                             3,582                       1,769   
4Q22                             3,392                       1,678   
3Q22          

In [48]:
startrows = 0
with pd.ExcelWriter('sample_output.xlsx', engine='xlsxwriter') as writer:
    for df in MSdataframes:
        df.to_excel(writer, index=False, sheet_name='Morgan_Stanley', startrow=startrows)
        startrows += df.shape[0] + 1

# Bank of America

In [49]:
BoAlinks = {"4Q23": "https://d1io3yog0oux5.cloudfront.net/_87c973d6eaec18dc169731b14cf06814/bankofamerica/db/806/9995/supplemental_information/The+Supplemental+Information_4Q23_ADA.pdf",\
        "2Q23": "https://d1io3yog0oux5.cloudfront.net/_87c973d6eaec18dc169731b14cf06814/bankofamerica/db/806/9921/supplemental_information/The+Supplemental+Information_2Q23_ADA.pdf", \
        "3Q22": "https://d1io3yog0oux5.cloudfront.net/_87c973d6eaec18dc169731b14cf06814/bankofamerica/db/806/9760/supplemental_information/The+Supplemental+Information+3Q22.pdf"}
bofa_search_words_with_threshold = [('Wealth Management',50),('Assets under management ',50),('Total assets',80) ]


In [50]:
results = []
source_docu = []
for index, link in BoAlinks.items():
    print(link)
    #texts = fileaccessing_builddb(link)
    texts = fileaccessing_builddb_pageselection(link, search_text_thresholds = bofa_search_words_with_threshold)
    quarters = Quarters_cal(currentQ = index)
    print(len(texts))
    ids = [str(i) for i in range(1, len(texts) + 1)]
    MSDB3 = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        ids=ids
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=MSDB3.as_retriever(search_kwargs={"k": 2}), # change database
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )
    query  = "Create a table to export values about  Global Wealth & Investment Management information, Assent under management, Net Cliet Flow, Total Wealth Advisors in "
    for quarter in quarters:
        query += f"{quarter}, "
    query += " In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE! \
    The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!"
    print(query)
    result = qa_chain(query)
    results.append(result['result'])
    source_docu.append(result['source_documents'])
    print("count before", MSDB3._collection.count())
    MSDB3._collection.delete(ids=ids)
    print("count after", MSDB3._collection.count())
    # check whether the database is cleaned or not.

https://d1io3yog0oux5.cloudfront.net/_87c973d6eaec18dc169731b14cf06814/bankofamerica/db/806/9995/supplemental_information/The+Supplemental+Information_4Q23_ADA.pdf
3
Create a table to export values about  Global Wealth & Investment Management information, Assent under management, Net Cliet Flow, Total Wealth Advisors in 4Q23, 3Q23, 4Q22,  In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE!     The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!


/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Sure, I can do that! Here's the table you requested:

| Quarters | Assets Under Management ($B) | Net Client Flows ($M) | Total Wealth Advisors |
| --- | --- | --- | --- |
| 4Q23 | 1,617.74 | 52.227 | 18,916 |
| 3Q23 | 1,601.47 | 20.785 | 19,273 |
| 4Q22 | 1,531.04 | 112.696 | 19,099 |

I hope this helps! Let me know if you have any questions or need further assistance.
count before 3
count after 0
https://d1io3yog0oux5.cloudfront.net/_87c973d6eaec18dc169731b14cf06814/bankofamerica/db/806/9921/supplemental_information/The+Supplemental+Information_2Q23_ADA.pdf
3
Create a table to export values about  Global Wealth & Investment Management information, Assent under management, Net Cliet Flow, Total Wealth Advisors in 2Q23, 1Q23, 2Q22,  In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE!     The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!
 Sure, I can do that! Here's the table you requested with the specified parameters:

| Quart

In [51]:
for i in results:
    print(i)

  Sure, I can do that! Here's the table you requested:

| Quarters | Assets Under Management ($B) | Net Client Flows ($M) | Total Wealth Advisors |
| --- | --- | --- | --- |
| 4Q23 | 1,617.74 | 52.227 | 18,916 |
| 3Q23 | 1,601.47 | 20.785 | 19,273 |
| 4Q22 | 1,531.04 | 112.696 | 19,099 |

I hope this helps! Let me know if you have any questions or need further assistance.
  Sure, I can do that! Here's the table you requested with the specified parameters:

| Quarters | Assets Under Management ($B) | Net Client Flows ($M) | Total Wealth Advisors |
| --- | --- | --- | --- |
| 2Q23 | 1,531 | 29 | 19,099 |
| 1Q23 | 1,411 | 16 | 18,449 |
| 2Q22 | 1,571 | 14 | 19,243 |

I hope this helps! Let me know if there's anything else I can assist you with.
  Sure, I can do that! Here's the table you requested:

| Quarters | Assets Under Management ($B) | Net Client Flows ($B) | Total Wealth Advisors |
| --- | --- | --- | --- |
| 3Q22 | 1.329 | 20.68 | 18,841 |
| 2Q22 | 1.578 | 44.69 | 18,855 |
| 3Q21

In [52]:
# Save the raw tables
pattern = r'\| Quarters \|.*?\|\n(.*?)\n\n'
BOAdataframes = []
for i in results:
    match = re.search(pattern, i, re.DOTALL)
    if match:
        table_str = match.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        
        headers = [header.strip() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
    
        df = pd.DataFrame(data, columns=headers)
        BOAdataframes.append(df)
   

In [67]:
# merge tables
pattern = r'\| Quarters \|.*?\|\n(.*?)\n\n'

tables = []
for i in results:
    match = re.search(pattern, i, re.DOTALL)
    if match:
        table_str = match.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        # Extract headers (first row) and data (remaining rows)
        headers = [header.strip().replace('($M)', '($B)').lower() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
combined_df = pd.concat(tables, ignore_index=True)
combined_df.drop(combined_df.tail(2).index, inplace=True)
subset_df = combined_df[combined_df['quarters'].isin(past_6Q(currentQ = "4Q23"))]
subset_df = subset_df.set_index('quarters')
subset_df = subset_df.reindex(index=past_6Q(currentQ = "4Q23")) 
BOAdataframes.append(subset_df)
print(subset_df)

         assets under management ($b) net client flows ($b)  \
quarters                                                      
4Q23                         1,617.74                52.227   
3Q23                         1,601.47                20.785   
2Q23                            1,531                    29   
1Q23                            1,411                    16   
4Q22                         1,531.04               112.696   
3Q22                            1.329                 20.68   
2Q22                            1,571                    14   

         total wealth advisors  
quarters                        
4Q23                    18,916  
3Q23                    19,273  
2Q23                    19,099  
1Q23                    18,449  
4Q22                    19,099  
3Q22                    18,841  
2Q22                    19,243  


In [76]:
with pd.ExcelWriter('sample_output.xlsx', engine='xlsxwriter') as writer:
    startrows = 0
    for df in MSdataframes:
        df.to_excel(writer, index=True, sheet_name='Morgan_Stanley', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in BOAdataframes:
        df.to_excel(writer, index=True, sheet_name='Bank_of_America', startrow=startrows)
        startrows += df.shape[0] + 1

# Goldman Sachs

In [77]:
GSlinks = {"4Q23": "https://www.goldmansachs.com/media-relations/press-releases/current/pdfs/2023-q4-results.pdf",\
        "2Q23": "https://www.goldmansachs.com/media-relations/press-releases/current/pdfs/2023-q2-results.pdf", \
        "3Q22": "https://www.goldmansachs.com/media-relations/press-releases/current/pdfs/2022-q3-results.pdf"}
gs_search_words_with_threshold=[('Assets Under Supervision',40),('Earnings Results',40),('AUS',40),('Wealth Management',50)]

In [78]:
results = []
for index, link in GSlinks.items():
    print(link)
    texts = fileaccessing_builddb_pageselection(link, search_text_thresholds = gs_search_words_with_threshold)
    quarters = Quarters_cal(currentQ = index)
    print(len(texts))
    ids = [str(i) for i in range(1, len(texts) + 1)]
    MSDB3 = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        ids=ids
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=MSDB3.as_retriever(search_kwargs={"k": 2}), # change database
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )
    query  = "Create a table to export values about containing wealth management information about Wealth AUS,\
 Wealth net flows, Wealth Management Fees, Net market appreciation / (depreciation)and Total Revenues,\
 Total AUS,about Wealth Management of "
    for quarter in quarters:
        query += f"{quarter}, "
    query += " In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE! \
    The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!"
    print(query)
    result = qa_chain(query)
    results.append(result['result'])
    print("count before", MSDB3._collection.count())
    MSDB3._collection.delete(ids=ids)
    print("count after", MSDB3._collection.count())
    # check whether the database is cleaned or not.

https://www.goldmansachs.com/media-relations/press-releases/current/pdfs/2023-q4-results.pdf
5
Create a table to export values about containing wealth management information about Wealth AUS, Wealth net flows, Wealth Management Fees, Net market appreciation / (depreciation)and Total Revenues, Total AUS,about Wealth Management of 4Q23, 3Q23, 4Q22,  In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE!     The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!


/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Sure, here's a table with the requested information:

| Quarters | Wealth AUS ($B) | Wealth Net Flows ($B) | Wealth Management Fees ($B) | Net Market Appreciation/Depreciation ($B) | Total Revenues ($B) | Total AUS ($B) |
| --- | --- | --- | --- | --- | --- | --- |
| 4Q23 | 2,547 | 33 | 2,812 | 141 | 10,277 | 2,812 |
| 3Q23 | 2,427 | 25 | 2,680 | 51 | 10,058 | 2,680 |
| 4Q22 | 2,378 | 19 | 2,547 | 87 | 9,879 | 2,547 |

Note: The values for Wealth AUS, Wealth Net Flows, Wealth Management Fees, Net Market Appreciation/Depreciation, and Total Revenues are in billions (B), while the value for Total AUS is in millions (M).
count before 5
count after 0
https://www.goldmansachs.com/media-relations/press-releases/current/pdfs/2023-q2-results.pdf
4
Create a table to export values about containing wealth management information about Wealth AUS, Wealth net flows, Wealth Management Fees, Net market appreciation / (depreciation)and Total Revenues, Total AUS,about Wealth Management of 2Q23, 1Q23, 2

In [79]:
for i in results:
    print(i)

  Sure, here's a table with the requested information:

| Quarters | Wealth AUS ($B) | Wealth Net Flows ($B) | Wealth Management Fees ($B) | Net Market Appreciation/Depreciation ($B) | Total Revenues ($B) | Total AUS ($B) |
| --- | --- | --- | --- | --- | --- | --- |
| 4Q23 | 2,547 | 33 | 2,812 | 141 | 10,277 | 2,812 |
| 3Q23 | 2,427 | 25 | 2,680 | 51 | 10,058 | 2,680 |
| 4Q22 | 2,378 | 19 | 2,547 | 87 | 9,879 | 2,547 |

Note: The values for Wealth AUS, Wealth Net Flows, Wealth Management Fees, Net Market Appreciation/Depreciation, and Total Revenues are in billions (B), while the value for Total AUS is in millions (M).
  Sure, here's your table with the specified parameters:

| QUARTERS |  Wealth AUS ($B) | Wealth Net Flows ($B) | Wealth Management Fees ($B) | Net Market Appreciation/Depreciation ($B) | Total Revenues ($B) | Total AUS ($B) |
| --- | --- | --- | --- | --- | --- | --- |
| 2Q23 | 2714 | 12 | 4 | 30 | 11 | 2714 |
| 1Q23 | 2672 | 57 | 8 | 68 | 10 | 2672 |
| 2Q22 | 2495 | -

In [80]:
# Save the raw tables
pattern1 = r'\| Quarters \|.*?\|\n(.*?)\n\n'
pattern2 = r'\| QUARTERS \|.*?\|\n(.*?)\n\n'
pattern3 = r'\| QUARTER \|.*?\|\n(.*?)\n\n'
GSdataframes = []
for i in results:
    match1 = re.search(pattern1, i, re.DOTALL)
    match2 = re.search(pattern2, i, re.DOTALL)
    match3 = re.search(pattern3, i, re.DOTALL)
    if match1:
        table_str = match1.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        
        headers = [header.strip() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
    
        df = pd.DataFrame(data, columns=headers)
        GSdataframes.append(df)
    if match2:
        table_str = match2.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        
        headers = [header.strip() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
    
        df = pd.DataFrame(data, columns=headers)
        GSdataframes.append(df)
    if match3:
        table_str = match3.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        
        headers = [header.strip() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
    
        df = pd.DataFrame(data, columns=headers)
        GSdataframes.append(df)
   

In [89]:
pattern1 = r'\| Quarters \|.*?\|\n(.*?)\n\n'
pattern2 = r'\| QUARTERS \|.*?\|\n(.*?)\n\n'
pattern3 = r'\| QUARTER \|.*?\|\n(.*?)\n\n'
tables = []

tables = []
for i in results:
    match1 = re.search(pattern1, i, re.DOTALL)
    match2 = re.search(pattern2, i, re.DOTALL)
    match3 = re.search(pattern3, i, re.DOTALL)
    if match1:
        table_str = match1.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        # Extract headers (first row) and data (remaining rows)
        headers = [header.strip().lower().replace('quarters', 'quarter') for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
    if match2:
        table_str = match2.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        # Extract headers (first row) and data (remaining rows)
        headers = [header.strip().lower().replace('quarters', 'quarter') for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
    if match3:
        table_str = match3.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        # Extract headers (first row) and data (remaining rows)
        headers = [header.strip().lower().replace('quarters', 'quarter') for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
combined_df = pd.concat(tables, ignore_index=True)
combined_df.drop(combined_df.tail(2).index, inplace=True)
subset_df = combined_df[combined_df['quarter'].isin(past_6Q(currentQ = "4Q23"))]
subset_df = subset_df.set_index('quarter')
subset_df = subset_df.reindex(index=past_6Q(currentQ = "4Q23"))
subset_df[subset_df.columns] = subset_df[subset_df.columns].replace('[\$,]', '', regex=True)
GSdataframes.append(subset_df)
print(subset_df)        

        wealth aus ($b) wealth net flows ($b) wealth management fees ($b)  \
quarter                                                                     
4Q23               2547                    33                        2812   
3Q23               2427                    25                        2680   
2Q23               2714                    12                           4   
1Q23               2672                    57                           8   
4Q22               2378                    19                        2547   
3Q22                667                    20                         2.8   
2Q22               2495                    -5                           2   

        net market appreciation/depreciation ($b) total revenues ($b)  \
quarter                                                                 
4Q23                                          141               10277   
3Q23                                           51               10058   
2Q23          

In [90]:
with pd.ExcelWriter('sample_output.xlsx', engine='xlsxwriter') as writer:
    startrows = 0
    for df in MSdataframes:
        df.to_excel(writer, index=True, sheet_name='Morgan_Stanley', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in BOAdataframes:
        df.to_excel(writer, index=True, sheet_name='Bank_of_America', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in GSdataframes:
        df.to_excel(writer, index=True, sheet_name='Goldenman_Sachs', startrow=startrows)
        startrows += df.shape[0] + 1

In [124]:
GSdataframes = GSdataframes[:4]

# JP Morgan

In [107]:
JPlinks = {"4Q23": "https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/quarterly-earnings/2023/4th-quarter/16d9371e-30e9-4898-abf6-d1f7c86fd311.pdf",\
        "2Q23": "https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/quarterly-earnings/2023/2nd-quarter/c9585f9b-75cc-4a49-b1ea-cec4423c87a7.pdf", \
        "3Q22": "https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/quarterly-earnings/2022/3rd-quarter/03e7493b-bc90-4ff5-8c1d-9e6611117091.pdf"}
jp_search_words_with_threshold=[('Asset & Wealth Management',80)]

In [108]:
results = []
for index, link in JPlinks.items():
    print(link)
    texts = fileaccessing_builddb_pageselection(link, search_text_thresholds = jp_search_words_with_threshold)
    quarters = Quarters_cal(currentQ = index)
    print(len(texts))
    ids = [str(i) for i in range(1, len(texts) + 1)]
    MSDB3 = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        ids=ids
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=MSDB3.as_retriever(search_kwargs={"k": 2}), # change database
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )
    query  = "Create a table to export values about containing wealth management information of these metrics: Total client assets, Net new assets\
    Total Assets under management and Assent under supervision,total,net revenue, net income,Asset management fees of "
    for quarter in quarters:
        query += f"{quarter}, "
    query += " In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE! \
    The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!"
    print(query)
    result = qa_chain(query)
    results.append(result['result'])
    print("count before", MSDB3._collection.count())
    MSDB3._collection.delete(ids=ids)
    print("count after", MSDB3._collection.count())
    # check whether the database is cleaned or not.

https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/quarterly-earnings/2023/4th-quarter/16d9371e-30e9-4898-abf6-d1f7c86fd311.pdf
3
Create a table to export values about containing wealth management information of these metrics: Total client assets, Net new assets    Total Assets under management and Assent under supervision,total,net revenue, net income,Asset management fees of 4Q23, 3Q23, 4Q22,  In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE!     The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!
 Sure, I can help you with that! Here's a table containing the wealth management information for the given metrics:

| Quarter | Total Client Assets ($B) | Net New Assets ($B) | Total Assets Under Management ($B) | Assets Under Supervision ($B) | Total Revenue ($B) | Net Income ($B) | Asset Management Fees ($B) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 4Q23 | 11.826 | 3.438 |

In [109]:
for i in results:
    print(i)

  Sure, I can help you with that! Here's a table containing the wealth management information for the given metrics:

| Quarter | Total Client Assets ($B) | Net New Assets ($B) | Total Assets Under Management ($B) | Assets Under Supervision ($B) | Total Revenue ($B) | Net Income ($B) | Asset Management Fees ($B) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 4Q23 | 11.826 | 3.438 | 19.827 | 5.241 | 5.095 | 1.217 | 3.137 |
| 3Q23 | 11.510 | 2.975 | 19.510 | 5.137 | 5.005 | 1.417 | 2.932 |
| 4Q22 | 10.698 | 2.782 | 18.627 | 4.943 | 4.588 | 1.134 | 2.825 |

Note: The values for Total Client Assets, Net New Assets, Total Assets Under Management, and Assets Under Supervision are in billions (B), while the rest of the values are in millions (M).
  I don't know how to create tables or export values. However, I can provide you with the information you need in a format that you can use to create your own table. Here are the values for the wealth management information for the specified m

In [128]:
# Save the raw tables
pattern = r'\| Quarter \|.*?\|\n(.*?)\n\n'
pattern_special = r'\| Quarter \| Metric \|.*?\|\n(.*?)\n\n'
JPMdataframes = []
for i in results:
    match = re.search(pattern, i, re.DOTALL)
    match_special = re.search(pattern_special, i, re.DOTALL)
    if match:
        if match_special:
            table_str = match.group(0)
            table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
            for table_row in table_rows:
                if table_row[1]!='  ':
                    quarter = table_row[1]
                table_row[1] = quarter
            headers = [header.strip() for header in table_rows[0] if header.strip()]
            data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]

            df = pd.DataFrame(data, columns=headers)
            JPMdataframes.append(df)
        else:
            table_str = match.group(0)
            table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]

            headers = [header.strip() for header in table_rows[0] if header.strip()]
            data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]

            df = pd.DataFrame(data, columns=headers)
            JPMdataframes.append(df)
print(JPMdataframes)

[  Quarter Total Client Assets ($B) Net New Assets ($B)  \
0     ---                      ---                 ---   
1    4Q23                   11.826               3.438   
2    3Q23                   11.510               2.975   
3    4Q22                   10.698               2.782   

  Total Assets Under Management ($B) Assets Under Supervision ($B)  \
0                                ---                           ---   
1                             19.827                         5.241   
2                             19.510                         5.137   
3                             18.627                         4.943   

  Total Revenue ($B) Net Income ($B) Asset Management Fees ($B)  
0                ---             ---                        ---  
1              5.095           1.217                      3.137  
2              5.005           1.417                      2.932  
3              4.588           1.134                      2.825  ,   Quarter Total Client Ass

In [144]:
pattern = r'\| Quarter \|.*?\|\n(.*?)\n\n'
pattern_special = r'\| Quarter \| Metric \|.*?\|\n(.*?)\n\n'
rename_col = {
    'Asset management fees': 'asset management fees (m)',
    'Assets under supervision': 'assets under supervision (b)',
    'Net income': 'net income (m)',
    'Net new assets': 'net new assets (m)',
    'Total Assets under management': 'total assets under management (b)',
    'Total client assets': 'total client assets (b)',
    'Total net revenue': 'total revenue (m)',
    'Quarter': 'quarter'
}
tables = []

tables = []
for i in results:
    match = re.search(pattern, i, re.DOTALL)
    match_special = re.search(pattern_special, i, re.DOTALL)
    if match:
        if match_special:
            table_str = match.group(0)
            table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
            for table_row in table_rows:
                if table_row[1]!='  ':
                    quarter = table_row[1]
                table_row[1] = quarter
            headers = [header.strip() for header in table_rows[0] if header.strip()]
            data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
            df = pd.DataFrame(data, columns=headers)
            pivot_df = df.pivot(index='Quarter', columns='Metric', values='Value')
            pivot_df.reset_index(inplace=True)
            pivot_df = pivot_df.rename(columns = rename_col)
            tables.append(pivot_df)
        else:
            table_str = match.group(0)
            table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
            # Extract headers (first row) and data (remaining rows)
            headers = [header.strip().lower().replace('$', '') for header in table_rows[0] if header.strip()]
            data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
            # Create DataFrame
            df = pd.DataFrame(data, columns=headers)
            tables.append(df)


In [152]:
combined_df = pd.concat(tables, ignore_index=True)
subset_df = combined_df[combined_df['quarter'].isin(past_6Q(currentQ = "4Q23"))]
subset_df.drop(subset_df.index[-2], inplace=True)
subset_df = subset_df.set_index('quarter')

/tmp/local/33310212/ipykernel_123894/2707125508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.drop(subset_df.index[-2], inplace=True)


In [153]:
subset_df = subset_df.reindex(index=past_6Q(currentQ = "4Q23"))
subset_df[subset_df.columns] = subset_df[subset_df.columns].replace('[\$,]', '', regex=True)
JPMdataframes.append(subset_df)
print(subset_df)

        total client assets (b) net new assets (b)  \
quarter                                              
4Q23                     11.826              3.438   
3Q23                     11.510              2.975   
2Q23                       3988                 14   
1Q23                       3511                 13   
4Q22                     10.698              2.782   
3Q22                  3773884 B                NaN   
2Q22                       3404                 10   

        total assets under management (b) assets under supervision (b)  \
quarter                                                                  
4Q23                               19.827                        5.241   
3Q23                               19.510                        5.137   
2Q23                                 4556                         4344   
1Q23                                 4103                         3522   
4Q22                               18.627                        4.94

In [154]:
with pd.ExcelWriter('sample_output.xlsx', engine='xlsxwriter') as writer:
    startrows = 0
    for df in MSdataframes:
        df.to_excel(writer, index=True, sheet_name='Morgan_Stanley', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in BOAdataframes:
        df.to_excel(writer, index=True, sheet_name='Bank_of_America', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in GSdataframes:
        df.to_excel(writer, index=True, sheet_name='Goldenman_Sachs', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in JPMdataframes:
        df.to_excel(writer, index=True, sheet_name='JPMorgan', startrow=startrows)
        startrows += df.shape[0] + 1

# Schwab

In [155]:
schlinks = {"4Q23": "https://content.schwab.com/web/retail/public/about-schwab/schw_q4_2023_earnings_release.pdf",
         "2Q23": "https://content.schwab.com/web/retail/public/about-schwab/schw_q2_2023_earnings_release.pdf"}
schwab_search_words_with_threshold = [('Total asset management',50),('Total client assets',80),('Net New Assets',30) ]

In [ ]:
results = []
for index, link in schlinks.items():
    print(link)
    texts = fileaccessing_builddb_pageselection(link, search_text_thresholds = schwab_search_words_with_threshold)
    quarters = Quarters_cal(currentQ = index)
    print(len(texts))
    ids = [str(i) for i in range(1, len(texts) + 1)]
    MSDB3 = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        ids=ids
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=MSDB3.as_retriever(search_kwargs={"k": 2}), # change database
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )
    query  = "Create a table containing Asset management information bout Total net new assets, \
    Net growth in client assets, Total Advice Solutions of "
    for quarter in quarters:
        query += f"{quarter}, "
    query += " In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE! \
    The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!"
    print(query)
    result = qa_chain(query)
    results.append(result['result'])
    print("count before", MSDB3._collection.count())
    MSDB3._collection.delete(ids=ids)
    print("count after", MSDB3._collection.count())
    # check whether the database is cleaned or not.

https://content.schwab.com/web/retail/public/about-schwab/schw_q4_2023_earnings_release.pdf
2
Create a table containing Asset management information bout Total net new assets,     Net growth in client assets, Total Advice Solutions of 4Q23, 3Q23, 4Q22,  In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE!     The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!


/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/home/zhaoshua/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Sure, here's a table containing asset management information about total net new assets, net growth in client assets, and total advice solutions for 4Q23, 3Q23, 4Q22, and M should be used instead of B:

| Quarter | Total Net New Assets (B) | Net Growth in Client Assets (B) | Total Advice Solutions (M) |
| --- | --- | --- | --- |
| 4Q23 | 43.8 | 21% | 126 |
| 3Q23 | 42.9 | 14% | 122 |
| 4Q22 | 41.5 | 6% | 119 |
| M |  | 115 | 

Note: In the table above, "B" represents billions of dollars, while "M" represents millions of dollars.
count before 2
count after 0
https://content.schwab.com/web/retail/public/about-schwab/schw_q2_2023_earnings_release.pdf
2
Create a table containing Asset management information bout Total net new assets,     Net growth in client assets, Total Advice Solutions of 2Q23, 1Q23, 2Q22,  In ONE TABLE using 'B' for billion and 'M' for million. Only ONE TABLE!     The Row names(y-axis) are QUARTERS(xQxx) and the Column names(x-axis) are METRICS name!
 I don't understa

In [165]:
for i in results:
    print(i)

  Sure, here's a table containing asset management information about total net new assets, net growth in client assets, and total advice solutions for 4Q23, 3Q23, 4Q22, and M should be used instead of B:

| Quarter | Total Net New Assets (B) | Net Growth in Client Assets (B) | Total Advice Solutions (M) |
| --- | --- | --- | --- |
| 4Q23 | 43.8 | 21% | 126 |
| 3Q23 | 42.9 | 14% | 122 |
| 4Q22 | 41.5 | 6% | 119 |
| M |  | 115 | 

Note: In the table above, "B" represents billions of dollars, while "M" represents millions of dollars.
  I don't understand what you want. Please clarify your request.


In [167]:
pattern = r'\| Quarter \|.*?\|\n(.*?)\n\n'
tables = []

tables = []
for i in results:
    match = re.search(pattern, i, re.DOTALL)
    if match:
        table_str = match.group(0)
        table_rows = [row.strip().split('|') for row in table_str.strip().split('\n')]
        # Extract headers (first row) and data (remaining rows)
        headers = [header.strip().lower() for header in table_rows[0] if header.strip()]
        data = [[cell.strip() for cell in row if cell.strip()] for row in table_rows[1:]]
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
combined_df = pd.concat(tables, ignore_index=True)
subset_df = combined_df[combined_df['quarter'].isin(Quarters_cal(currentQ = "4Q23"))]
print(subset_df)
   

  quarter total net new assets (b) net growth in client assets (b)  \
1    4Q23                     43.8                             21%   
2    3Q23                     42.9                             14%   
3    4Q22                     41.5                              6%   

  total advice solutions (m)  
1                        126  
2                        122  
3                        119  


In [168]:
with pd.ExcelWriter('sample_output.xlsx', engine='xlsxwriter') as writer:
    startrows = 0
    for df in MSdataframes:
        df.to_excel(writer, index=True, sheet_name='Morgan_Stanley', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in BOAdataframes:
        df.to_excel(writer, index=True, sheet_name='Bank_of_America', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in GSdataframes:
        df.to_excel(writer, index=True, sheet_name='Goldenman_Sachs', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    for df in JPMdataframes:
        df.to_excel(writer, index=True, sheet_name='JPMorgan', startrow=startrows)
        startrows += df.shape[0] + 1
    startrows = 0
    subset_df.to_excel(writer, index=True, sheet_name='Schwab', startrow=startrows)